In [ ]:
import os
import pandas as pd


In [1]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



